In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images

Dataset URL: https://www.kaggle.com/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images
License(s): CC0-1.0
 87% 86.0M/98.7M [00:01<00:00, 102MB/s]
100% 98.7M/98.7M [00:01<00:00, 95.5MB/s]


In [4]:
from zipfile import ZipFile
file_name = "/content/melanoma-skin-cancer-dataset-of-10000-images.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential# type: ignore
from tensorflow.keras.layers import Dense, Dropout# type: ignore
from tensorflow.keras.applications import VGG16# type: ignore
from tensorflow.keras.layers import Dense, Flatten# type: ignore
from tensorflow.keras.models import Model# type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator# type: ignore
from PIL import Image
import matplotlib.pyplot as plt


In [6]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)


In [8]:
model = Model(inputs=base_model.input, outputs=x)

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [10]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

In [13]:
train_generator = train_datagen.flow_from_directory("/content/melanoma_cancer_dataset/train", target_size=(224, 224),batch_size=32, class_mode='binary')

Found 9605 images belonging to 2 classes.


In [26]:
history = model.fit(train_generator, epochs=8, steps_per_epoch=100, validation_data=test_generator)

Epoch 1/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 47s 445ms/step - accuracy: 0.8899 - loss: 0.2719 - val_accuracy: 0.9080 - val_loss: 0.2359
Epoch 2/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 44s 447ms/step - accuracy: 0.8807 - loss: 0.2803 - val_accuracy: 0.9080 - val_loss: 0.2546
Epoch 3/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 415ms/step - accuracy: 0.8764 - loss: 0.2859 - val_accuracy: 0.9000 - val_loss: 0.2762
Epoch 4/8
  1/100 ━━━━━━━━━━━━━━━━━━━━ 11s 121ms/step - accuracy: 0.9062 - loss: 0.1783

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.9062 - loss: 0.1783 - val_accuracy: 0.8980 - val_loss: 0.3056
Epoch 5/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 451ms/step - accuracy: 0.8808 - loss: 0.2886 - val_accuracy: 0.9080 - val_loss: 0.2471
Epoch 6/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 44s 445ms/step - accuracy: 0.8933 - loss: 0.2726 - val_accuracy: 0.9040 - val_loss: 0.2313
Epoch 7/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 42s 419ms/step - accuracy: 0.8852 - loss: 0.2583 - val_accuracy: 0.8940 - val_loss: 0.2929
Epoch 8/8
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.8750 - loss: 0.3643 - val_accuracy: 0.8950 - val_loss: 0.2827


In [27]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(r"/content/melanoma_cancer_dataset/test", target_size=(224, 224),
                                                  batch_size=32, class_mode='binary')


Found 1000 images belonging to 2 classes.


In [28]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - accuracy: 0.8910 - loss: 0.2928
Test accuracy: 0.89
